# The Drivers of Errors in Single Unit Zestimates at Zillow
***

## Introduction

### Author
- Samuel Davila
 - Data Scientist
 - Zillow DS Department

### Purpose of project
- Identify what causes the accuracy of our estimates of property value ("Zestimates") to increase or decrease. 

### Why is this project important?
- By understanding what improves and what weakens the accuracy of our estimates, we can shore up our weaknesses while maintaining our strengths. 

### Data source
- Single unit property data from Zillow table in Data Science Database
***

## Executive Summary

### Goals
- Improve original estimate of the log error by using clustering methodologies.
    - Identify drivers of log error
    - Create model that...
    

- Deliver the following:

    - zillow_clustering_project.ipynb

    - README.md

    - acquire.py

    - prep.py

    - preprocessing.py

    - model.py
    
    - A presentation that walks through each step of our project and this notebook as a whole.

### Analysis

Analysis goes here

### Recommendation

Recommendation goes here

### Expectation

Expectation goes here
***

## Project Planning

### Acquire

### Preparation

### Exploraton

### Model

### Conclusion
***

## Acquire
Acquire the data we need for our project from the zillow table in the data science database.

Create __acquire.py__ file that contains the functions needed to replicate this process.
***

#### Importing modules needed for code in notebook to run.

In [1]:
# set up environment
from acquire import get_zillow_data
from prep import drop_missing_columns
from prep import missing_rows
from prep import drop_selected_columns
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

#### Using function in acquire.py file to import data then previewing data.

In [2]:
# create variable that will hold DF for easy access to data
df = get_zillow_data()

# previewing data
df.head()

,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,...,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,id,parcelid,logerror,transactiondate
0,2288172,12177905,NaN,None,None,3.0,4.0,NaN,8.0,3.0,...,2016.0,36225.0,1777.51,None,NaN,6.037300e+13,3,12177905,-0.103410,2017-01-01
1,1970746,10887214,1.0,None,None,3.0,3.0,NaN,8.0,3.0,...,2016.0,45726.0,1533.89,None,NaN,6.037124e+13,4,10887214,0.006940,2017-01-01
2,781532,12095076,1.0,None,None,3.0,4.0,NaN,9.0,3.0,...,2016.0,496619.0,9516.26,None,NaN,6.037461e+13,6,12095076,-0.001011,2017-01-01
3,870991,12069064,NaN,None,None,1.0,2.0,NaN,5.0,1.0,...,2016.0,199662.0,2366.08,None,NaN,6.037302e+13,7,12069064,0.101723,2017-01-01
4,1246926,12790562,NaN,None,None,3.0,4.0,NaN,9.0,3.0,...,2016.0,43056.0,3104.19,None,NaN,6.037500e+13,8,12790562,-0.040966,2017-01-02


### Acquire Takeaways
- Acquiring data from zillow database on the data science database server using the __get_zillow_data__ function
- Function needed to replicate this phase is located in the __acquire.py__ file

## Prepare
Prepare, tidy, and clean the data for exploration and analysis.

Create __prep.py__ file that contains the functions needed to replicate this process.
***

#### We'll use .info to see null value counts, data types, and row / columns count.

In [3]:
# using info function to examine data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47414 entries, 0 to 47413
Data columns (total 63 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            47414 non-null  int64  
 1   parcelid                      47414 non-null  int64  
 2   airconditioningtypeid         22070 non-null  float64
 3   architecturalstyletypeid      0 non-null      object 
 4   basementsqft                  0 non-null      object 
 5   bathroomcnt                   47414 non-null  float64
 6   bedroomcnt                    47414 non-null  float64
 7   buildingclasstypeid           8 non-null      float64
 8   buildingqualitytypeid         46923 non-null  float64
 9   calculatedbathnbr             47368 non-null  float64
 10  decktypeid                    0 non-null      object 
 11  finishedfloor1squarefeet      0 non-null      object 
 12  calculatedfinishedsquarefeet  47407 non-null  float64
 13  f

- Many columns have a large amount of null values
    - We'll be using a function (missing_rows) to examine the amount of null values in more depth later
    - After examining in-depth, we'll decide how to proceed


- Data type of several columns need to be converted 
    - We'll handle this in a later stage since the columns in question may be removed for in an alternate step 
        - For example, we may remove a column with a bad data type due to it having too many nulls


- Several columns, such as rawcensustractandblock, are categorical variables that may have a very large amount of unique values.
    - Encoding every value for these types of columns may be computationally expensive and add a large amount of columns to our dataset.
        - We will need to set a cutoff amount for categorical column unique values and remove any columns that exceed this amount.


- Once we've identified which columns to move into explore with, we'll need to rename them if they are hard to read, such as landtaxvaluedollarcnt.


#### Using missing_rows function from prep.py to create DF that shows the total number and percent of missing rows in each column of our data.

In [4]:
# passing dataframe to function from prep.py file
missing_rows(df)

,num_rows_missing,pct_rows_missing
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,25344,53.452567
architecturalstyletypeid,47414,100.000000
basementsqft,47414,100.000000
...,...,...
censustractandblock,116,0.244653
id,0,0.000000
parcelid,0,0.000000
logerror,0,0.000000


- There are many columns with a substantial amount of missing values
    - We will remove any columns that are missing 40% or more of their values
    - We don't have an exact formula for why we should go with 40% but it seems reasonable to remove any columns that are missing that proportion of values.

#### Using drop_missing_columns function from prep.py to remove columns that are missing 40% or more of their values.

In [5]:
# function removes any columns that are missing 40% or more of their values
drop_missing_columns(df)

,id,parcelid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,fips,fullbathcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,id,parcelid,logerror,transactiondate
0,2288172,12177905,3.0,4.0,8.0,3.0,2376.0,2376.0,6037.0,3.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,3,12177905,-0.103410,2017-01-01
1,1970746,10887214,3.0,3.0,8.0,3.0,1312.0,1312.0,6037.0,3.0,...,73681.0,119407.0,2016.0,45726.0,1533.89,6.037124e+13,4,10887214,0.006940,2017-01-01
2,781532,12095076,3.0,4.0,9.0,3.0,2962.0,2962.0,6037.0,3.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,6,12095076,-0.001011,2017-01-01
3,870991,12069064,1.0,2.0,5.0,1.0,738.0,738.0,6037.0,1.0,...,18890.0,218552.0,2016.0,199662.0,2366.08,6.037302e+13,7,12069064,0.101723,2017-01-01
4,1246926,12790562,3.0,4.0,9.0,3.0,3039.0,3039.0,6037.0,3.0,...,177527.0,220583.0,2016.0,43056.0,3104.19,6.037500e+13,8,12790562,-0.040966,2017-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47409,2864704,10833991,3.0,3.0,8.0,3.0,1741.0,1741.0,6037.0,3.0,...,265000.0,379000.0,2016.0,114000.0,4685.34,6.037132e+13,77608,10833991,-0.002245,2017-09-20
47410,673515,11000655,2.0,2.0,6.0,2.0,1286.0,1286.0,6037.0,2.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,77609,11000655,0.020615,2017-09-20
47411,1843709,12773139,1.0,3.0,4.0,1.0,1032.0,1032.0,6037.0,1.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,77611,12773139,0.037129,2017-09-21
47412,1187175,12826780,2.0,3.0,6.0,2.0,1762.0,1762.0,6037.0,2.0,...,140000.0,522000.0,2016.0,382000.0,6317.15,6.037503e+13,77612,12826780,0.007204,2017-09-25


#### While examining the data in SQL, we noticed that several columns appeared to have identical values. We're going to see how many values are unique between them.

In [6]:
# comparing each pair of columns we suspected are duplicates
# the resulting variables tell us how many rows differ between each
sqft_columns_diff = (df.finishedsquarefeet12 != df.calculatedfinishedsquarefeet).sum()
bathroom_count_diff = (df.calculatedbathnbr != df.bathroomcnt).sum()
bathroom_count_diff_alt = (df.fullbathcnt != df.bathroomcnt).sum()

print(f'Number of different values between finishedsquarefeet12 and calculatedfinishedsquarefeet: {sqft_columns_diff}')
print(f'Number of different values between calculatedbathnbr and bathroomcnt: {bathroom_count_diff}')
print(f'Number of different values between fullbathcnt and bathroomcnt: {bathroom_count_diff_alt}')

Number of different values between finishedsquarefeet12 and calculatedfinishedsquarefeet: 48
Number of different values between calculatedbathnbr and bathroomcnt: 46
Number of different values between fullbathcnt and bathroomcnt: 46


- The sum of non-duplicate values between all of these columns is 140 (48 + 46 + 46). 
    - This is roughly .003 of all rows in the DF
        - We can safely drop all the following majority-duplicate columns and only lose a extremely small proportion of rows.
            - finishedsquarefeet12
            - calculatedbathnbr
            - fullbathcnt
        - We could drop any of their alternativee columns instead but it would cost us time to find a non-arbitrary reason to do so and given how few unique values we're losing, the loss is relatively inconsequential.
        - These columns will be dropped in an upcoming function, __drop_selected_columns__, along with any other columns that are found to be in need of removal.

#### While examining the data in SQL, I noticed that tax_value_dollarcnt appeared to be the sum of landtaxvaluedollarcnt and structuretaxvaluedollarcnt. 

#### To test this I'm going to combine landtaxvaluedollarcnt and structuretaxvaluedollarcnt manually, then compare the results to landtaxvaluedollarcount and see how many of the values match.

In [7]:
# creating new df that holds all three columns we're interested in
tax_eval_df = df[['structuretaxvaluedollarcnt', 'landtaxvaluedollarcnt', 'taxvaluedollarcnt']]

# creating new column in df that is the sum of landtaxvaluedollarcnt and structuretaxvaluedollarcnt
tax_eval_df['taxvaluedollarcnt_test'] = df.structuretaxvaluedollarcnt + df.landtaxvaluedollarcnt

# comparing taxvaluedollarcnt to our manually calculated column and finding the average % of rows where the values matched
(tax_eval_df.taxvaluedollarcnt_test == tax_eval_df.taxvaluedollarcnt).mean()

0.9986501877082719

- 99% of the our manually summed values matched the original 
    - Safe the say that in a vast majority of rows, taxvaluedollarcnt is the sum of landtaxvaluedollarcnt and structuretaxvaluedollarcnt
        - This being the case, we're only going to keep taxvaluedollarcnt and remove the other two columns since their values are already accounted for in this column.
        - If need be, we can add them back later and see if we get better results by having them seperated.
            - The columns will be dropped at a later step using the __drop_selected_columns__ function

#### We're using nunique() to see how many unique values each column has. This is useful for identifying categorical columns with large amounts of unique values and columns with only a single value.

In [8]:
# nunique() displays each column and the amount of unique values that it holds
df.nunique()

id                              47293
parcelid                        47293
bathroomcnt                        13
bedroomcnt                         12
buildingqualitytypeid              12
calculatedbathnbr                  12
calculatedfinishedsquarefeet     4302
finishedsquarefeet12             4300
fips                                3
fullbathcnt                        12
heatingorsystemtypeid               3
latitude                        38668
longitude                       37041
lotsizesquarefeet               16506
propertycountylandusecode          40
propertylandusetypeid              10
propertyzoningdesc               1854
rawcensustractandblock          25213
regionidcity                      135
regionidcounty                      3
regionidzip                       290
roomcnt                             5
unitcnt                             1
yearbuilt                         131
structuretaxvaluedollarcnt      28782
taxvaluedollarcnt               32671
assessmentye

- The following are categorical columns will be removed because they contain more than 100 unique values. Encoding them would be computationaly expensive and add a vast amount of columns to our dataframe.

    - id (both)
    - parcelid (both)
    - latitude
    - longitude
    - propertycountylandusecode
    - propertyzoningdesc
    - rawcensustractandblock
    - regionidcity
    - regionidzip
    - yearbuilt
    - censustractandblock
    - transactiondate

    
- There are ways to avoid the consequences of encoding categorical columns with lots of features, but in the interest of time we will avoid these routes for now.


- The following columns will be removed because they only contain 1 unique value and would thus not allow us to make any meaningful distinctions with them.

    - assessmentyear
    - unitcnt


- All of these columns will be removed using the __drop_selected_columns__ function from prep.py

#### Dropping columns that meet any of the following criteria (identified in steps prior)
    - Categorical with 100+ unique values 
    - Only contain 1 unique value
    - landtaxvaluedollarcnt and structuretaxvaluedollarcnt
        - summed under taxvaluedollarcnt column
    - Near-duplicates of alternate column(s)

In [9]:
# using function from prep.py to drop columns meeting any criterion above
drop_selected_columns(df)

In [10]:
df.nunique()

bathroomcnt                        13
bedroomcnt                         12
buildingqualitytypeid              12
calculatedfinishedsquarefeet     4302
fips                                3
heatingorsystemtypeid               3
lotsizesquarefeet               16506
propertylandusetypeid              10
regionidcounty                      3
roomcnt                             5
taxvaluedollarcnt               32671
taxamount                       46062
logerror                        47020
dtype: int64

## Explore

## Modeling

## Overall Conclusion and Takeaways